In [1]:
from collections import namedtuple

from gym.wrappers import RescaleAction, TimeLimit
import numpy as np
import pandas as pd
from stable_baselines3 import TD3
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from tqdm import tqdm

from environments import ARESEASequential

In [2]:
def load_sequential(model_name, max_episode_steps=50, measure_beam="us"):

    ModelSetup = namedtuple("ModelSetup", ["name","env","model","max_episode_steps","measure_beam"])

    log_dir = f"models/{model_name}"

    def make_env():
        env = ARESEASequential(
            backend="machine",
            backendargs={"measure_beam": measure_beam}
        )
        env = TimeLimit(env, max_episode_steps=max_episode_steps)
        env = RescaleAction(env, -1, 1)
        return env

    env = DummyVecEnv([make_env])
    env = VecNormalize.load(f"{log_dir}/vec_normalize.pkl", env)
    env.training = False
    env.norm_reward = False

    model = TD3.load(f"{log_dir}/model")

    return ModelSetup(model_name, env, model, max_episode_steps, measure_beam)

In [10]:
def pack_dataframe(fn):
    def wrapper(setup, desired):
        observations, rewards, beam_images = fn(setup, desired)
        observations = np.array(observations)

        df = pd.DataFrame(np.arange(len(observations)), columns=["step"])
        df["q1"] = observations[:,0]
        df["q2"] = observations[:,1]
        df["cv"] = observations[:,2]
        df["q3"] = observations[:,3]
        df["ch"] = observations[:,4]
        df["mup_x"] = observations[:,5]
        df["mup_y"] = observations[:,6]
        df["sigmap_x"] = observations[:,7]
        df["sigmap_y"] = observations[:,8]
        df["mu_x"] = observations[:,9]
        df["mu_y"] = observations[:,10]
        df["sigma_x"] = observations[:,11]
        df["sigma_y"] = observations[:,12]
        df["reward"] = [np.nan] + rewards
        df["beam_image"] = beam_images

        df["model_name"] = setup.name
        df["max_episode_steps"] = setup.max_episode_steps
        df["measure_beam"] = setup.measure_beam

        return df
    
    return wrapper

In [11]:
@pack_dataframe
def run(setup, desired):
    env, model = setup.env, setup.model

    env.get_attr("unwrapped")[0].next_initial = np.zeros(5)
    env.get_attr("unwrapped")[0].next_desired = desired

    observations = []
    rewards = []
    beam_images = []

    observation = env.reset()
    observations.append(env.unnormalize_obs(observation).squeeze())
    beam_images.append(env.get_attr("backend")[0].last_beam_image)

    with tqdm(total=setup.max_episode_steps) as pbar:
        done = False
        while not done:
            action, _ = model.predict(observation, deterministic=True)
            observation, reward, done, info = env.step(action)

            observations.append(env.unnormalize_obs(observation).squeeze())
            rewards.append(reward.squeeze())
            beam_images.append(env.get_attr("backend")[0].last_beam_image)

            pbar.update(1)

    observations[-1] = env.unnormalize_obs(info[0]["terminal_observation"].squeeze())

    return observations, rewards, beam_images

In [12]:
setup = load_sequential("amber-mountain-976")

In [13]:
df = run(setup, np.zeros(4))

100%|██████████| 50/50 [07:14<00:00,  8.68s/it]


,step,q1,q2,cv,q3,ch,mup_x,mup_y,sigmap_x,sigmap_y,mu_x,mu_y,sigma_x,sigma_y,reward,beam_image,model_name,max_episode_steps,measure_beam
0,0,-8.423128e-08,8.022620e-09,1.958620e-08,1.964785e-14,6.518805e-13,6.019531e-14,-2.626839e-13,4.671420e-12,-4.424122e-12,-2.191068e-03,3.180970e-04,0.000350,0.000411,NaN,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",amber-mountain-976,50,us
1,1,-1.278085e+00,2.920598e+00,3.000000e+00,-1.214446e-04,5.999974e-04,6.019531e-14,-2.626839e-13,4.671420e-12,-4.424122e-12,-9.494628e-04,3.034156e-04,0.000186,0.000582,0.36849228,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",amber-mountain-976,50,us
2,2,-4.205281e+00,5.516942e+00,5.994088e+00,-3.000994e-04,1.076514e-03,6.019531e-14,-2.626839e-13,4.671420e-12,-4.424122e-12,-2.058276e-04,4.893800e-05,0.000096,0.000490,0.6697913,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",amber-mountain-976,50,us
3,3,-6.357828e+00,7.190644e+00,8.364227e+00,-3.508193e-04,6.845113e-04,6.019531e-14,-2.626839e-13,4.671420e-12,-4.424122e-12,2.257464e-04,-1.125574e-04,0.000023,0.000353,0.26966745,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",amber-mountain-976,50,us
4,4,-7.643978e+00,7.363797e+00,9.814283e+00,-2.847884e-04,1.113440e-04,6.019531e-14,-2.626839e-13,4.671420e-12,-4.424122e-12,1.726296e-04,-1.468140e-04,0.000023,0.000266,0.19535112,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",amber-mountain-976,50,us
5,5,-7.708496e+00,5.925903e+00,1.104495e+01,-2.015532e-04,-2.090680e-04,6.019531e-14,-2.626839e-13,4.671420e-12,-4.424122e-12,-1.062336e-04,-9.298220e-05,0.000023,0.000254,0.17663863,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",amber-mountain-976,50,us
6,6,-7.243416e+00,4.272390e+00,1.237370e+01,-1.444104e-04,-9.251322e-05,6.019531e-14,-2.626839e-13,4.671420e-12,-4.424122e-12,-1.062336e-04,-2.446900e-05,0.000023,0.000254,0.09561317,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",amber-mountain-976,50,us
7,7,-6.763373e+00,2.689986e+00,1.369690e+01,-1.300711e-04,1.631229e-05,6.019531e-14,-2.626839e-13,4.671420e-12,-4.424122e-12,-2.655840e-05,-9.787600e-06,0.000023,0.000249,0.16293322,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",amber-mountain-976,50,us
8,8,-6.342036e+00,1.260316e+00,1.464613e+01,-1.276819e-04,2.470362e-05,6.019531e-14,-2.626839e-13,4.671420e-12,-4.424122e-12,-1.327920e-05,-1.468140e-05,0.000023,0.000245,0.02920192,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",amber-mountain-976,50,us
9,9,-6.082750e+00,2.698187e-01,1.521941e+01,-1.205976e-04,5.052481e-06,6.019531e-14,-2.626839e-13,4.671420e-12,-4.424122e-12,-2.655840e-05,-1.468140e-05,0.000023,0.000237,0.005954224,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",amber-mountain-976,50,us
